In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import torch
from torchsummary import summary
import pickle
import os
from functions import Dataset,prototypical_loss as loss_fn ,ProtoNet#,PrototypicalBatchSampler

### Waist

In [ ]:
#waistDF = pickle.load( open( "waist_final_10272800_new.pkl", "rb" ))
waistDF = pickle.load( open( "waist_final_10272800_new_nozero.pkl", "rb" ))

XTrain = waistDF[['x','y','z']]
YTrain = waistDF[['speed']]

X_trainWaist = XTrain.values.reshape(102728,100,3)
Y_trainWaist = YTrain.values.reshape(102728,100,1)
Y_trainWaist = np.array([float(max(z)) for z in Y_trainWaist])

waistTrain_x, waistVal_x, waistTrain_y, waistVal_y = train_test_split(X_trainWaist, Y_trainWaist, test_size = 0.1,random_state=100)
waistTest_x, waistVal_x, waistTest_y, waistVal_y = train_test_split(waistVal_x, waistVal_y, test_size = 0.5,random_state=100)
print(len(waistTrain_x))
print(len(waistTest_x))
print(len(waistVal_x))

92455
5136
5137


In [ ]:
waistTrain_x = waistTrain_x.reshape(92455,1,100,3)
waistTest_x = waistTest_x.reshape(5136,1,100,3)
waistVal_x = waistVal_x.reshape(5137,1,100,3)
print(waistTrain_x.shape)#,train_y.shape)
print(waistTest_x.shape)#,test_y.shape)
print(waistVal_x.shape)#,val_y.shape)

(92455, 1, 100, 3)
(5136, 1, 100, 3)
(5137, 1, 100, 3)


In [ ]:
waistTrain_x = torch.from_numpy(waistTrain_x)
waistTrain_y = torch.from_numpy(waistTrain_y)

waistTest_x = torch.from_numpy(waistTest_x)
waistTest_y = torch.from_numpy(waistTest_y)

waistVal_x = torch.from_numpy(waistVal_x)
waistVal_y = torch.from_numpy(waistVal_y)

### Wrist

In [ ]:
#wristDF = pickle.load( open( "wrist_final_10272800_new.pkl", "rb" ))
wristDF = pickle.load( open( "wrist_final_10272800_new_nozero.pkl", "rb" ))

XTrain = wristDF[['x','y','z']]
YTrain = wristDF[['speed']]

X_trainWrist = XTrain.values.reshape(102728,100,3)
Y_trainWrist = YTrain.values.reshape(102728,100,1)
Y_trainWrist = np.array([float(max(z)) for z in Y_trainWrist])

wristTrain_x, wristVal_x, wristTrain_y, wristVal_y = train_test_split(X_trainWrist, Y_trainWrist, test_size = 0.1,random_state=100)
wristTest_x, wristVal_x, wristTest_y, wristVal_y = train_test_split(wristVal_x, wristVal_y, test_size = 0.5,random_state=100)
print(len(wristTrain_x))
print(len(wristTest_x))
print(len(wristVal_x))

92455
5136
5137


In [ ]:
wristTrain_x = wristTrain_x.reshape(92455,1,100,3)
wristTest_x = wristTest_x.reshape(5136,1,100,3)
wristVal_x = wristVal_x.reshape(5137,1,100,3)
print(wristTrain_x.shape)#,train_y.shape)
print(wristTest_x.shape)#,test_y.shape)
print(wristVal_x.shape)#,val_y.shape)

(92455, 1, 100, 3)
(5136, 1, 100, 3)
(5137, 1, 100, 3)


In [ ]:
wristTrain_x = torch.from_numpy(wristTrain_x)
wristTrain_y = torch.from_numpy(wristTrain_y)

wristTest_x = torch.from_numpy(wristTest_x)
wristTest_y = torch.from_numpy(wristTest_y)

wristVal_x = torch.from_numpy(wristVal_x)
wristVal_y = torch.from_numpy(wristVal_y)

In [ ]:
#Classes
print(len(np.unique(wristTrain_y)),len(np.unique(wristTest_y)),len(np.unique(wristVal_y)))

14 14 14


### DataSet Class

In [ ]:
training_set = Dataset(wristTrain_x,waistTrain_x, wristTrain_y)
test_set = Dataset(wristTest_x,waistTest_x, wristTest_y)
val_set = Dataset(wristVal_x,waistVal_x, wristVal_y)

In [ ]:
x = ['dataset_root','experiment_root','cuda']

classes_per_it_tr = 3
classes_per_it_val =3
classes_per_it_tr = 3

manual_seeds =7
iterations = 10
epochs = 100
learning_rate = 0.01
lr_scheduler_step = 0.1
lr_scheduler_gamma = 0.5
cuda = 0
num_support_tr = 40
num_query_tr = 40
num_support_val = 40
num_query_val = 40

In [ ]:
class PrototypicalBatchSampler(object):
    '''
    PrototypicalBatchSampler: yield a batch of indexes at each iteration.
    Indexes are calculated by keeping in account 'classes_per_it' and 'num_samples',
    In fact at every iteration the batch indexes will refer to  'num_support' + 'num_query' samples
    for 'classes_per_it' random classes.

    __len__ returns the number of episodes per epoch (same as 'self.iterations').
    '''

    def __init__(self, labels, classes_per_it, num_samples, iterations):
        '''
        Initialize the PrototypicalBatchSampler object
        Args:
        - labels: an iterable containing all the labels for the current dataset
        samples indexes will be infered from this iterable.
        - classes_per_it: number of random classes for each iteration
        - num_samples: number of samples for each iteration for each class (support + query)
        - iterations: number of iterations (episodes) per epoch
        '''
        super(PrototypicalBatchSampler, self).__init__()
        self.labels = labels
        self.labels = [float(x) for x in self.labels]
        self.classes_per_it = classes_per_it
        self.sample_per_class = num_samples
        self.iterations = iterations

        self.classes, self.counts = np.unique(self.labels, return_counts=True)
        self.classes = [float(x) for x in self.classes]
        self.classes = torch.FloatTensor(self.classes)
        #print(self.classes)
        # create a matrix, indexes, of dim: classes X max(elements per class)
        # fill it with nans
        # for every class c, fill the relative row with the indices samples belonging to c
        # in numel_per_class we store the number of samples for each class/row
        self.idxs = range(len(self.labels))
        self.indexes = np.empty((len(self.classes), max(self.counts)), dtype=int) * np.nan
        self.indexes = torch.Tensor(self.indexes)
        self.numel_per_class = torch.zeros_like(self.classes)
        for idx, label in enumerate(self.labels):

            label_idx = np.argwhere(self.classes == label).item()
            self.indexes[label_idx, np.where(np.isnan(self.indexes[label_idx]))[0][0]] = idx
            self.numel_per_class[label_idx] += 1

    def __iter__(self):
        '''
        yield a batch of indexes
        '''
        
        spc = self.sample_per_class
        cpi = self.classes_per_it
        
        #print(spc)
        for it in range(self.iterations):
            batch_size = spc * cpi
            batch = torch.LongTensor(batch_size)
            c_idxs = torch.randperm(len(self.classes))[:cpi]
            for i, c in enumerate(self.classes[c_idxs]):
                s = slice(i * spc, (i + 1) * spc)
                # FIXME when torch.argwhere will exists
                label_idx = torch.arange(len(self.classes)).long()[self.classes == c].item()
                sample_idxs = torch.randperm(int(self.numel_per_class[label_idx]))[:spc]
                batch[s] = self.indexes[label_idx][sample_idxs]
            batch = batch[torch.randperm(len(batch))]
            yield batch

    def __len__(self):
        '''
        returns the number of iterations (episodes) per epoch
        '''
        return self.iterations

In [ ]:
def init_seed(manual_seed):
    '''
    Disable cudnn to maximize reproducibility
    '''
    torch.cuda.cudnn_enabled = False
    np.random.seed(manual_seed)
    torch.manual_seed(manual_seed)
    torch.cuda.manual_seed(manual_seed)
    
def init_dataset(mode):
    if (mode == "train"):dataset = training_set
    if (mode == "test"):dataset = test_set
    if (mode == "val"):dataset = val_set
    n_classes = 14
    if n_classes < classes_per_it_tr or n_classes < classes_per_it_val:
        raise(Exception('There are not enough classes in the dataset in order ' +
                        'to satisfy the chosen classes_per_it. Decrease the ' +
                        'classes_per_it_{tr/val} option and try again.'))
    return dataset

def init_sampler(labels, mode):
    if 'train' in mode:
        classes_per_it = classes_per_it_tr
        num_samples = num_support_tr + num_query_tr
    else:
        classes_per_it = classes_per_it_val
        num_samples = num_support_val + num_query_val

    return PrototypicalBatchSampler(labels=labels,classes_per_it=classes_per_it,
                                    num_samples=num_samples,
                                    iterations=iterations)

def init_dataloader(mode):
    dataset = init_dataset(mode)
    sampler = init_sampler(dataset.Y, mode)
    dataloader = torch.utils.data.DataLoader(dataset, batch_sampler=sampler)
    return dataloader

In [ ]:
def init_protonet():
    device = 'cuda:0' if torch.cuda.is_available() and cuda else 'cpu'
    model = ProtoNet().to(device)
    return model

def init_optim(model):return torch.optim.Adam(params=model.parameters(),lr=learning_rate)

def init_lr_scheduler(optim):
    return torch.optim.lr_scheduler.StepLR(optimizer=optim,gamma=lr_scheduler_gamma,step_size=lr_scheduler_step)

def save_list_to_file(path, thelist):
    with open(path, 'w') as f:
        for item in thelist:f.write("%s\n" % item)

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class Combine_Waist_Wrist(nn.Module):
    def __init__(self, modelA, modelB):
        super(Combine_Waist_Wrist, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        
        self.fc1 = nn.Linear(100*2*4*2,64)
        #self.fc1 = nn.Linear(100*2*2,64)
        
        self.classifier = nn.Linear(64,1)   ### 2 -> 1 , make labels 1 dimensions: 0 or 1
        #linear layers
        #linear layers
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        out = torch.cat((x1, x2), dim=1)
        
        out = F.relu(self.fc1(out))
        #out = (self.fc1(out))
        x = self.classifier(out)  # not relu
        #x = F.tanh(out)
        return x

In [ ]:
def train(tr_dataloader, model, optim, lr_scheduler, val_dataloader=None):
    
    device = 'cuda:0' if torch.cuda.is_available() and cuda else 'cpu'

    if val_dataloader is None:best_state = None
    
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    best_acc = 0

    best_model_path = 'best_model.pth'
    last_model_path = 'last_model.pth'

    for epoch in range(epochs):
        print('=== Epoch: {} ==='.format(epoch))
        tr_iter = iter(tr_dataloader)
        model.train()
        for batch in tr_iter:
            optim.zero_grad()
            x1,x2, y = batch
            x1,x2, y = x1.to(device), x2.to(device),y.to(device)
            model_output = model(x1.float(),x2.float())
            loss,acc= loss_fn(model_output,target=y,n_support=num_support_tr)
            loss.backward()
            optim.step()
            train_loss.append(loss.item())
            train_acc.append(acc.item())
        avg_loss = np.mean(train_loss[-iterations:])
        avg_acc = np.mean(train_acc[-iterations:])
        print('Avg Train Loss: {}, Avg Train Acc: {}'.format(avg_loss, avg_acc))
        lr_scheduler.step()
        if val_dataloader is None:
            continue
            
        val_iter = iter(val_dataloader)
        model.eval()
        
        for batch in val_iter:
            x1,x2, y = batch
            x1,x2, y = x1.to(device), x2.to(device),y.to(device)
            model_output = model(x1.float(),x2.float())
            loss, acc = loss_fn(model_output, target=y,n_support=num_support_val)
            val_loss.append(loss.item())
            val_acc.append(acc.item())
        avg_loss = np.mean(val_loss[-iterations:])
        avg_acc = np.mean(val_acc[-iterations:])
        postfix = ' (Best)' if avg_acc >= best_acc else ' (Best: {})'.format(best_acc)
        print('Avg Val Loss: {}, Avg Val Acc: {}{}'.format(avg_loss, avg_acc, postfix))
        if avg_acc >= best_acc:
            torch.save(model.state_dict(), best_model_path)
            best_acc = avg_acc
            best_state = model.state_dict()

    torch.save(model.state_dict(), last_model_path)

    for name in ['train_loss', 'train_acc', 'val_loss', 'val_acc']:
        save_list_to_file(os.path.join(name + '.txt'), locals()[name])

    return best_state, best_acc, train_loss, train_acc, val_loss, val_acc


def test(test_dataloader, model):
    '''
    Test the model trained with the prototypical learning algorithm
    '''
    device = 'cuda:0' if torch.cuda.is_available() and cuda else 'cpu'
    avg_acc = list()
    for epoch in range(10):
        test_iter = iter(test_dataloader)
        for batch in test_iter:
            x1,x2, y = batch
            x1,x2, y = x1.to(device), x2.to(device),y.to(device)
            model_output = model(x1.float(),x2.float())
            _, acc = loss_fn(model_output, target=y,n_support=num_support_val)
            avg_acc.append(acc.item())
    avg_acc = np.mean(avg_acc)
    print('Test Acc: {}'.format(avg_acc))

    return avg_acc


def eval():
    '''
    Initialize everything and train
    '''
    #options = get_parser().parse_args()

    if torch.cuda.is_available() and not options.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

    init_seed()
    test_dataloader = init_dataset()[-1]
    model = init_protonet()
    model_path = os.path.join('best_model.pth')
    model.load_state_dict(torch.load(model_path))

    test(opt=options,test_dataloader=test_dataloader,model=model)


In [ ]:
def main():
  
    if torch.cuda.is_available() and not cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

    init_seed(manual_seeds)
    tr_dataloader = init_dataloader('train')
    val_dataloader = init_dataloader('val')
    # trainval_dataloader = init_dataloader(options, 'trainval')
    test_dataloader = init_dataloader('test')

    #model = init_protonet()
    modelWaist = init_protonet()
    modelWrist = init_protonet()
    modelCombine = Combine_Waist_Wrist(modelWaist,modelWrist)
    optim = init_optim(modelCombine)
    
    lr_scheduler = init_lr_scheduler(optim)
    res = train(tr_dataloader=tr_dataloader,val_dataloader=val_dataloader,lr_scheduler=lr_scheduler,
                model=modelCombine,optim=optim)
    
    best_state, best_acc, train_loss, train_acc, val_loss, val_acc = res
    print('Testing with last model..')
    test(test_dataloader=test_dataloader,model=modelCombine)

    modelCombine.load_state_dict(best_state)
    print('Testing with best model..')
    test(test_dataloader=test_dataloader,model=modelCombine)

    # optim = init_optim(options, model)
    # lr_scheduler = init_lr_scheduler(options, optim)

    # print('Training on train+val set..')
    # train(opt=options,tr_dataloader=trainval_dataloader,
    #       val_dataloader=None,model=model,optim=optim,lr_scheduler=lr_scheduler)

    # print('Testing final model..')
    # test(opt=options,test_dataloader=test_dataloader,model=model)

In [ ]:
classes_per_it_tr = 3
classes_per_it_val =3
classes_per_it_tr = 3

iterations = 10
epochs = 100

num_support_tr = 40
num_query_tr = 40
num_support_val = 40
num_query_val = 40

main()

=== Epoch: 0 ===
Avg Train Loss: 1.1443148255348206, Avg Train Acc: 0.5883333355188369
Avg Val Loss: 0.7787049114704132, Avg Val Acc: 0.6225000113248825 (Best)
=== Epoch: 1 ===
Avg Train Loss: 0.64212886095047, Avg Train Acc: 0.6716666638851165
Avg Val Loss: 0.6412241250276566, Avg Val Acc: 0.7366666674613953 (Best)
=== Epoch: 2 ===
Avg Train Loss: 0.6531057864427566, Avg Train Acc: 0.7508333325386047
Avg Val Loss: 0.5308374851942063, Avg Val Acc: 0.7599999994039536 (Best)
=== Epoch: 3 ===
Avg Train Loss: 0.5599259823560715, Avg Train Acc: 0.760833340883255
Avg Val Loss: 0.6483340293169022, Avg Val Acc: 0.7258333325386047 (Best: 0.7599999994039536)
=== Epoch: 4 ===
Avg Train Loss: 0.6311839878559112, Avg Train Acc: 0.7166666626930237
Avg Val Loss: 0.6300419420003891, Avg Val Acc: 0.7166666686534882 (Best: 0.7599999994039536)
=== Epoch: 5 ===
Avg Train Loss: 0.511501032114029, Avg Train Acc: 0.776666671037674
Avg Val Loss: 0.45612139999866486, Avg Val Acc: 0.8299999952316284 (Best)
=== 

Avg Train Loss: 0.46107910126447677, Avg Train Acc: 0.7825000047683716
Avg Val Loss: 0.4499838650226593, Avg Val Acc: 0.85 (Best: 0.8816666781902314)
=== Epoch: 47 ===
Avg Train Loss: 0.5878714770078659, Avg Train Acc: 0.8008333265781402
Avg Val Loss: 0.46063866317272184, Avg Val Acc: 0.8233333349227905 (Best: 0.8816666781902314)
=== Epoch: 48 ===
Avg Train Loss: 0.4600615620613098, Avg Train Acc: 0.8358333349227905
Avg Val Loss: 0.481922821700573, Avg Val Acc: 0.7866666674613952 (Best: 0.8816666781902314)
=== Epoch: 49 ===
Avg Train Loss: 0.35651449859142303, Avg Train Acc: 0.860833328962326
Avg Val Loss: 0.3997245579957962, Avg Val Acc: 0.8308333396911621 (Best: 0.8816666781902314)
=== Epoch: 50 ===
Avg Train Loss: 0.3327326849102974, Avg Train Acc: 0.8791666686534881
Avg Val Loss: 0.28911475017666816, Avg Val Acc: 0.8791666626930237 (Best: 0.8816666781902314)
=== Epoch: 51 ===
Avg Train Loss: 0.4511170119047165, Avg Train Acc: 0.8024999976158143
Avg Val Loss: 0.4412672400474548, Avg

Avg Val Loss: 0.4872275717556477, Avg Val Acc: 0.8341666638851166 (Best: 0.8908333301544189)
=== Epoch: 92 ===
Avg Train Loss: 0.30118299648165703, Avg Train Acc: 0.8733333230018616
Avg Val Loss: 0.5928711771965027, Avg Val Acc: 0.7566666692495346 (Best: 0.8908333301544189)
=== Epoch: 93 ===
Avg Train Loss: 0.36218438670039177, Avg Train Acc: 0.8441666662693024
Avg Val Loss: 0.35378574058413503, Avg Val Acc: 0.8549999952316284 (Best: 0.8908333301544189)
=== Epoch: 94 ===
Avg Train Loss: 0.42437812946736814, Avg Train Acc: 0.8025000095367432
Avg Val Loss: 0.40537002980709075, Avg Val Acc: 0.8174999952316284 (Best: 0.8908333301544189)
=== Epoch: 95 ===
Avg Train Loss: 0.33703594356775285, Avg Train Acc: 0.8633333265781402
Avg Val Loss: 0.3377784192562103, Avg Val Acc: 0.8558333337306976 (Best: 0.8908333301544189)
=== Epoch: 96 ===
Avg Train Loss: 0.3021284569054842, Avg Train Acc: 0.8816666662693023
Avg Val Loss: 0.46581166684627534, Avg Val Acc: 0.7991666674613953 (Best: 0.8908333301544